# Naive Bayes text classification

Naive Bayes model with TF/IDF algorithm to solve text classification problem


## Term Frequency
TF — это частотность термина, которая измеряет, насколько часто термин встречается в документе. Логично предположить, что в длинных документах термин может встретиться в больших количествах, чем в коротких, поэтому абсолютные числа тут не катят. Поэтому применяют относительные — делят количество раз, когда нужный термин встретился в тексте, на общее количество слов в тексте. 

## Inverse Document Frequency
IDF — это обратная частотность документов. Она измеряет непосредственно важность термина. То есть, когда мы считали TF, все термины считаются как бы равными по важности друг другу. Но всем известно, что, например, предлоги встречаются очень часто, хотя практически не влияют на смысл текста. И что с этим поделать? Ответ прост — посчитать IDF. Он считается как логарифм от общего количества документов, делённого на количество документов, в которых встречается термин а.

#### TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)
#### IDF термина а = (Общее количество документов / Количество документов, в которых встречается термин а)

In [1]:
import math
from collections import Counter
from collections import defaultdict
import numpy as np

In [36]:
class NaiveBayes:
    def __init__(self, data):
        self.corpus, self.labels  = self.data_split(data)
        self.classes = Counter(self.labels)
     
    def data_split(self, data):
        corpus = []
        labels = []
        for text, label in data:
            corpus.append(text.split())
            labels.append(label)
        return corpus, labels
    
    def get_unique_words(self):
        unique_words_in_corpus = Counter()
        for doc in self.corpus:
            unique_words_in_corpus += Counter(doc)
        return len(unique_words_in_corpus)
    
    def get_class_words(self):
        total = defaultdict(int)
        words = defaultdict(Counter)       
        for i in range(len(self.corpus)):           
            words[self.labels[i]] += Counter(self.corpus[i])
            total[self.labels[i]] += len(self.corpus[i])
        return words, total
    
    def get_prior (self, class_label):
        return self.classes[class_label]/len(self.corpus)          
    
    def get_p (self, class_label, word):
        p = (self.words[class_label][word] + 1)/(self.total[class_label] + self.unique)
        return p #math.log(p)
    
    def fit(self):
        self.words, self.total = self.get_class_words()
        self.unique = self.get_unique_words()
            
    def predict_doc(self, doc):
        p = dict()
        for label in self.classes:
            p[label] = self.get_prior (label)
            for word in doc.split():
                p[label] *= self.get_p(label,word)
        y = max(p, key=p.get)
        return y, p 
    
    def predict(self, docs):
        matches = np.zeros(len(docs))
        for doc, label in docs:
            y,p = self.predict_doc(doc)
            matches[i] = (y == label)
        return matches
            

In [37]:
# Train data
data = [["Chinese Beijing Chinese","0"],
        ["Chinese Chinese Shanghai","0"],
        ["Chinese Macao","0"],
        ["Tokyo Japan Chinese","1"]]
pred =  "Chinese Chinese Chinese Tokyo Japan"

nbm = NaiveBayes(data)
nbm.fit()
y,p = nbm.predict_doc(pred)
print(y,p)

# model
#nbm = NaiveBayes(data_train)
#nbm.fit()
#matches = nbm.predict(data_test)
#print ("Accuracy: ", matches.mean()) 

0 {'0': 0.00030121377997263036, '1': 0.00013548070246744226}


In [ ]:
# data.csv



In [ ]:
# Must return[ ('Chinese Chinese Chinese Tokyo Japan', '0')]
# pobability {'1': 0.00013548070246744226, '0': 0.00030121377997263036}
# or log     {'1': -7.906681345001262, '0': -7.10769031284391}


In [27]:
# TF-IDF

def calc_tf(docum):
    """
    parameters: docum - List of words
    returns: Counter object with TF for all words in docum
    """
    tf = Counter(docum)
    for i in tf:
        tf[i] = tf[i]/float(len(docum))
    return tf

def calc_idf(word, corpus):
    """
    parameters: corpus - List of texts
                word - or which to calculate IDF
    returns: value, idf for word in corpus            
    """
    word_in_doc_count = sum([1.0 for i in corpus if word in i])
    idf = word_in_doc_count/len(corpus)
    return idf
        
def calc_tfidf(corpus):
    docs_list = []
    for doc in corpus:
        tf_idf_dict = {}
        tf = calc_tf(doc)
        for word in tf:
            tf_idf_dict[word] = tf[word] / calc_idf(word, corpus)
        docs_list.append(tf_idf_dict)
    return docs_list

texts = [['pasta', 'la', 'vista', 'baby', 'la', 'vista'], 
         ['hasta', 'siempre', 'comandante', 'baby', 'la', 'siempre'], 
         ['siempre', 'comandante', 'baby', 'la', 'siempre']]
print (calc_tfidf(texts))




[{'pasta': 0.5, 'la': 0.3333333333333333, 'vista': 1.0, 'baby': 0.16666666666666666}, {'hasta': 0.5, 'siempre': 0.5, 'comandante': 0.25, 'baby': 0.16666666666666666, 'la': 0.16666666666666666}, {'siempre': 0.6000000000000001, 'comandante': 0.30000000000000004, 'baby': 0.2, 'la': 0.2}]


- http://nlpx.net/archives/57
- https://stevenloria.com/tf-idf/
- https://ru.wikipedia.org/wiki/TF-IDF